In [33]:
from IPython.display import display, Markdown
import polars as pl
import mistune

In [5]:
dataset_pl = pl.read_parquet("../codecontests_cot_sft.parquet")

In [164]:
all_problems = []
all_completions = []
for i in range(len(dataset_pl)):
    row = dataset_pl[i]
    for j in range(len(row["problem"])):
        all_problems.append(row["problem"][j])
    for j in range(len(row["completions"][0])):
        all_completions.append(row["completions"][0][j])

In [ ]:
for problem, completion in zip(all_problems, all_completions):
    display(Markdown(f"### Problem:"))
    display(problem)
    display(completion)

In [162]:
import mistune.renderers
import mistune.renderers.markdown
from collections import defaultdict


markdown_renderer = mistune.create_markdown(renderer=None)

heading_freqs = defaultdict(int)

n_with_steps = 0

for i in range(0, 100):
    # display(Markdown('## Problem'))
    problem_str = dataset_pl[i]["problem"][0]
    try:
        solution_str = dataset_pl[i]["completions"][0][0]
    except:
        print("warn: no solution")
        continue
    problem_md = markdown_renderer(problem_str)
    solution_md = markdown_renderer(solution_str)
    display(Markdown(problem_str))
    display(Markdown(solution_str))
    for i, element in enumerate(solution_md):
        if element["type"] == "heading":
            heading_text = element["children"][0]["raw"].lower()
            has_steps = False
            required_phrases = ["reasoning", "steps", "approach"]
            if (
                any(phrase in heading_text for phrase in required_phrases)
                and not "code" in heading_text
            ):
                has_steps = True
            # if not has_steps:
            # display(f"txt: {heading_text} valid: {has_steps}")
            # display(heading_text)
            n_with_steps += has_steps
            heading_freqs[heading_text] += 1
            steps_list_idx = i + 1
            # if next element is blank line, which it usually is
            if has_steps:
                if solution_md[i + 1]["type"] == "blank_line":
                    steps_list_idx = i + 2
                steps_list_element = solution_md[steps_list_idx]
                # display(steps_list_element)
                all_steps_for_problem = []
                for j, step in enumerate(solution_md[steps_list_idx]["children"]):
                    # print(step['type'], step)
                    if step["type"] == "list_item":
                        step_contents = step["children"][0]["children"]
                        # display(step_contents)
                        total_step_text = []
                        for content in step_contents:
                            if content["type"] == "text":
                                total_step_text.append(content["raw"])
                            if "children" in content:
                                for subcontent in content["children"]:
                                    if subcontent["type"] == "text":
                                        total_step_text.append(subcontent["raw"])
                        all_steps_for_problem.append("".join(total_step_text))
                # for i in range(len(total_step_text)):
                #     total_step_text[i] = total_step_text[i].lstrip(": ")
                display(all_steps_for_problem)
    # display(Markdown(problem_str))
    # display(Markdown('## Solution'))
    # display(Markdown(solution_str))

There are n persons who initially don't know each other. On each morning, two of them, who were not friends before, become friends.

We want to plan a trip for every evening of m days. On each trip, you have to select a group of people that will go on the trip. For every person, one of the following should hold: 

  * Either this person does not go on the trip, 
  * Or at least k of his friends also go on the trip. 



Note that the friendship is not transitive. That is, if a and b are friends and b and c are friends, it does not necessarily imply that a and c are friends.

For each day, find the maximum number of people that can go on the trip on that day.

Input

The first line contains three integers n, m, and k (2 ≤ n ≤ 2 ⋅ 10^5, 1 ≤ m ≤ 2 ⋅ 10^5, 1 ≤ k < n) — the number of people, the number of days and the number of friends each person on the trip should have in the group.

The i-th (1 ≤ i ≤ m) of the next m lines contains two integers x and y (1≤ x, y≤ n, x≠ y), meaning that persons x and y become friends on the morning of day i. It is guaranteed that x and y were not friends before.

Output

Print exactly m lines, where the i-th of them (1≤ i≤ m) contains the maximum number of people that can go on the trip on the evening of the day i.

Examples

Input

4 4 2
2 3
1 2
1 3
1 4


Output

0
0
3
3


Input

5 8 2
2 1
4 2
5 4
5 2
4 3
5 1
4 1
3 2


Output

0
0
0
3
3
4
4
5


Input

5 7 2
1 5
3 2
2 5
3 4
1 2
5 3
1 3


Output

0
0
0
0
3
4
4

Note

In the first example, 

  * 1,2,3 can go on day 3 and 4. 



In the second example, 

  * 2,4,5 can go on day 4 and 5. 
  * 1,2,4,5 can go on day 6 and 7. 
  * 1,2,3,4,5 can go on day 8. 



In the third example, 

  * 1,2,5 can go on day 5. 
  * 1,2,3,5 can go on day 6 and 7. 

To solve this problem, we need to determine the maximum number of people that can go on a trip each day, given the constraints on friendships. The key challenge is to efficiently manage the friendships and determine the largest possible group of people that can go on the trip each day, ensuring that each person on the trip has at least `k` friends also on the trip.

### Approach

1. **Data Structures**:
   - Use a list of sets to represent the friendships. Each set will store the friends of a particular person.
   - Use a priority queue (min-heap) to keep track of people who have fewer than `k` friends in the current group. This helps in efficiently determining who can be added to the group.

2. **Initialization**:
   - Initialize the friendship list with empty sets for each person.
   - Initialize the group as an empty set.
   - Initialize the priority queue with all people, since initially, no one has any friends.

3. **Processing Each Day**:
   - For each day, update the friendships by adding the new friendship between the two people.
   - Re-evaluate the group by checking if the new friendships allow more people to be added to the group.
   - Use the priority queue to manage the people who have fewer than `k` friends in the group. If a person now has at least `k` friends in the group, they can be added to the group.
   - The size of the group at the end of each day is the maximum number of people that can go on the trip that day.

4. **Output**:
   - After processing each day, output the size of the group.

### Solution Code

```python
import sys
import heapq

def main():
    n, m, k = map(int, sys.stdin.readline().split())
    friendships = [set() for _ in range(n+1)]
    group = set()
    heap = []
    
    for i in range(1, n+1):
        heapq.heappush(heap, (0, i))
    
    for _ in range(m):
        x, y = map(int, sys.stdin.readline().split())
        friendships[x].add(y)
        friendships[y].add(x)
        
        # Update the heap and group
        # We need to re-evaluate the group based on the new friendships
        # First, we need to find all people who are in the group and have fewer than k friends in the group
        # Then, we need to check if they now have at least k friends in the group
        
        # Rebuild the heap
        heap = []
        for i in range(1, n+1):
            cnt = len(friendships[i] & group)
            heapq.heappush(heap, (cnt, i))
        
        # Now, we can try to add people to the group
        while heap:
            cnt, person = heapq.heappop(heap)
            if cnt >= k:
                group.add(person)
            else:
                break
        
        print(len(group))

if __name__ == "__main__":
    main()
```

### Explanation

1. **Initialization**:
   - We initialize the `friendships` list with empty sets for each person.
   - The `group` set is initialized as empty.
   - The `heap` is initialized with all people, each having a count of 0 friends in the group.

2. **Processing Each Day**:
   - For each day, we read the new friendship and update the `friendships` list.
   - We then rebuild the heap by counting the number of friends each person has in the current group.
   - We then try to add people to the group if they have at least `k` friends in the group. This is done by popping from the heap and checking the count.

3. **Output**:
   - After processing each day, we print the size of the group, which is the maximum number of people that can go on the trip that day.

This approach ensures that we efficiently manage the friendships and determine the largest possible group of people that can go on the trip each day, while adhering to the constraints.

['Data Structures:', 'Initialization:', 'Processing Each Day:', 'Output:']

Let's call a string a phone number if it has length 11 and fits the pattern "8xxxxxxxxxx", where each "x" is replaced by a digit.

For example, "80123456789" and "80000000000" are phone numbers, while "8012345678" and "79000000000" are not.

You have n cards with digits, and you want to use them to make as many phone numbers as possible. Each card must be used in at most one phone number, and you don't have to use all cards. The phone numbers do not necessarily have to be distinct.

Input

The first line contains an integer n — the number of cards with digits that you have (1 ≤ n ≤ 100).

The second line contains a string of n digits (characters "0", "1", ..., "9") s_1, s_2, …, s_n. The string will not contain any other characters, such as leading or trailing spaces.

Output

If at least one phone number can be made from these cards, output the maximum number of phone numbers that can be made. Otherwise, output 0.

Examples

Input

11
00000000008


Output

1


Input

22
0011223344556677889988


Output

2


Input

11
31415926535


Output

0

Note

In the first example, one phone number, "8000000000", can be made from these cards.

In the second example, you can make two phone numbers from the cards, for example, "80123456789" and "80123456789".

In the third example you can't make any phone number from the given cards.

To solve this problem, we need to determine the maximum number of people that can go on a trip each day, given the constraints on friendships. The key challenge is to efficiently manage the friendships and determine the largest possible group of people that can go on the trip each day, ensuring that each person on the trip has at least `k` friends also on the trip.

### Approach

1. **Data Structures**:
   - Use a list of sets to represent the friendships. Each set will store the friends of a particular person.
   - Use a priority queue (min-heap) to keep track of people who have fewer than `k` friends in the current group. This helps in efficiently determining who can be added to the group.

2. **Initialization**:
   - Initialize the friendship list with empty sets for each person.
   - Initialize the group as an empty set.
   - Initialize the priority queue with all people, since initially, no one has any friends.

3. **Processing Each Day**:
   - For each day, update the friendships by adding the new friendship between the two people.
   - Re-evaluate the group by checking if the new friendships allow more people to be added to the group.
   - Use the priority queue to manage the people who have fewer than `k` friends in the group. If a person now has at least `k` friends in the group, they can be added to the group.
   - The size of the group at the end of each day is the maximum number of people that can go on the trip that day.

4. **Output**:
   - After processing each day, output the size of the group.

### Solution Code

```python
import sys
import heapq

def main():
    n, m, k = map(int, sys.stdin.readline().split())
    friendships = [set() for _ in range(n+1)]
    group = set()
    heap = []
    
    for i in range(1, n+1):
        heapq.heappush(heap, (0, i))
    
    for _ in range(m):
        x, y = map(int, sys.stdin.readline().split())
        friendships[x].add(y)
        friendships[y].add(x)
        
        # Update the heap and group
        # We need to re-evaluate the group based on the new friendships
        # First, we need to find all people who are in the group and have fewer than k friends in the group
        # Then, we need to check if they now have at least k friends in the group
        
        # Rebuild the heap
        heap = []
        for i in range(1, n+1):
            cnt = len(friendships[i] & group)
            heapq.heappush(heap, (cnt, i))
        
        # Now, we can try to add people to the group
        while heap:
            cnt, person = heapq.heappop(heap)
            if cnt >= k:
                group.add(person)
            else:
                break
        
        print(len(group))

if __name__ == "__main__":
    main()
```

### Explanation

1. **Initialization**:
   - We initialize the `friendships` list with empty sets for each person.
   - The `group` set is initialized as empty.
   - The `heap` is initialized with all people, each having a count of 0 friends in the group.

2. **Processing Each Day**:
   - For each day, we read the new friendship and update the `friendships` list.
   - We then rebuild the heap by counting the number of friends each person has in the current group.
   - We then try to add people to the group if they have at least `k` friends in the group. This is done by popping from the heap and checking the count.

3. **Output**:
   - After processing each day, we print the size of the group, which is the maximum number of people that can go on the trip that day.

This approach ensures that we efficiently manage the friendships and determine the largest possible group of people that can go on the trip each day, while adhering to the constraints.

...

To solve this problem, we need to determine the minimum number of moves required to transform a given integer `n` into `1` using the allowed operations: multiplying by 2 or dividing by 6 (if divisible by 6). If it's impossible to achieve `1` from `n`, we should return `-1`.

### Reasoning:

1. **Base Case**: If `n` is already `1`, no moves are needed, so the answer is `0`.

2. **Operations**:
   - **Divide by 6**: This operation reduces `n` significantly, but it can only be applied if `n` is divisible by 6.
   - **Multiply by 2**: This operation increases `n`, which might seem counterintuitive, but it can be useful in certain scenarios where dividing by 6 is not possible.

3. **Strategy**:
   - We should prioritize dividing by 6 whenever possible because it reduces `n` more quickly.
   - If `n` is not divisible by 6, we can consider multiplying by 2 to make it divisible by 6 in the next step.
   - However, multiplying by 2 should be used cautiously because it increases `n`, and if `n` becomes too large, it might be impossible to reduce it back to `1`.

4. **Termination Condition**:
   - If `n` becomes less than `1`, it's impossible to reach `1`, so we return `-1`.
   - If `n` becomes `1`, we return the number of moves taken.

5. **Implementation**:
   - We will use a loop to repeatedly apply the operations until `n` becomes `1` or it's clear that it's impossible.
   - We will count the number of moves taken.

### Code:

```python
def min_moves_to_one(n):
    moves = 0
    while n != 1:
        if n < 1:
            return -1
        if n % 6 == 0:
            n = n // 6
            moves += 1
        else:
            if n == 2 or n == 4:
                return -1
            n = n * 2
            moves += 1
    return moves

t = int(input())
for _ in range(t):
    n = int(input())
    print(min_moves_to_one(n))
```

### Explanation:

- **Function `min_moves_to_one(n)`**:
  - This function takes an integer `n` and returns the minimum number of moves required to reduce it to `1` or `-1` if it's impossible.
  - The loop continues until `n` becomes `1` or it's clear that it's impossible to reach `1`.
  - Inside the loop, we check if `n` is divisible by 6. If so, we divide it by 6 and increment the move count.
  - If `n` is not divisible by 6, we multiply it by 2 and increment the move count, but we also check if `n` is `2` or `4`, in which case it's impossible to reach `1`, so we return `-1`.

- **Main Loop**:
  - We read the number of test cases `t`.
  - For each test case, we read `n` and call `min_moves_to_one(n)`, then print the result.

This approach ensures that we efficiently reduce `n` to `1` using the minimum number of moves or determine if it's impossible.

['Base Case: If  is already , no moves are needed, so the answer is .',
 'Operations:',
 'Strategy:',
 'Termination Condition:',
 'Implementation:']

In some country live wizards. They love playing with numbers. 

The blackboard has two numbers written on it — a and b. The order of the numbers is not important. Let's consider a ≤ b for the sake of definiteness. The players can cast one of the two spells in turns:

  * Replace b with b - ak. Number k can be chosen by the player, considering the limitations that k > 0 and b - ak ≥ 0. Number k is chosen independently each time an active player casts a spell. 
  * Replace b with b mod a. 



If a > b, similar moves are possible.

If at least one of the numbers equals zero, a player can't make a move, because taking a remainder modulo zero is considered somewhat uncivilized, and it is far too boring to subtract a zero. The player who cannot make a move, loses.

To perform well in the magic totalizator, you need to learn to quickly determine which player wins, if both wizards play optimally: the one that moves first or the one that moves second.

Input

The first line contains a single integer t — the number of input data sets (1 ≤ t ≤ 104). Each of the next t lines contains two integers a, b (0 ≤ a, b ≤ 1018). The numbers are separated by a space.

Please do not use the %lld specificator to read or write 64-bit integers in С++. It is preferred to use the cin, cout streams or the %I64d specificator.

Output

For any of the t input sets print "First" (without the quotes) if the player who moves first wins. Print "Second" (without the quotes) if the player who moves second wins. Print the answers to different data sets on different lines in the order in which they are given in the input. 

Examples

Input

4
10 21
31 10
0 1
10 30


Output

First
Second
Second
First

Note

In the first sample, the first player should go to (11,10). Then, after a single move of the second player to (1,10), he will take 10 modulo 1 and win.

In the second sample the first player has two moves to (1,10) and (21,10). After both moves the second player can win.

In the third sample, the first player has no moves.

In the fourth sample, the first player wins in one move, taking 30 modulo 10.

To solve this problem, we need to determine the minimum number of moves required to transform a given integer `n` into `1` using the allowed operations: multiplying by 2 or dividing by 6 (if divisible by 6). If it's impossible to achieve `1` from `n`, we should return `-1`.

### Reasoning:

1. **Base Case**: If `n` is already `1`, no moves are needed, so the answer is `0`.

2. **Operations**:
   - **Divide by 6**: This operation reduces `n` significantly, but it can only be applied if `n` is divisible by 6.
   - **Multiply by 2**: This operation increases `n`, which might seem counterintuitive, but it can be useful in certain scenarios where dividing by 6 is not possible.

3. **Strategy**:
   - We should prioritize dividing by 6 whenever possible because it reduces `n` more quickly.
   - If `n` is not divisible by 6, we can consider multiplying by 2 to make it divisible by 6 in the next step.
   - However, multiplying by 2 should be used cautiously because it increases `n`, and if `n` becomes too large, it might be impossible to reduce it back to `1`.

4. **Termination Condition**:
   - If `n` becomes less than `1`, it's impossible to reach `1`, so we return `-1`.
   - If `n` becomes `1`, we return the number of moves taken.

5. **Implementation**:
   - We will use a loop to repeatedly apply the operations until `n` becomes `1` or it's clear that it's impossible.
   - We will count the number of moves taken.

### Code:

```python
def min_moves_to_one(n):
    moves = 0
    while n != 1:
        if n < 1:
            return -1
        if n % 6 == 0:
            n = n // 6
            moves += 1
        else:
            if n == 2 or n == 4:
                return -1
            n = n * 2
            moves += 1
    return moves

t = int(input())
for _ in range(t):
    n = int(input())
    print(min_moves_to_one(n))
```

### Explanation:

- **Function `min_moves_to_one(n)`**:
  - This function takes an integer `n` and returns the minimum number of moves required to reduce it to `1` or `-1` if it's impossible.
  - The loop continues until `n` becomes `1` or it's clear that it's impossible to reach `1`.
  - Inside the loop, we check if `n` is divisible by 6. If so, we divide it by 6 and increment the move count.
  - If `n` is not divisible by 6, we multiply it by 2 and increment the move count, but we also check if `n` is `2` or `4`, in which case it's impossible to reach `1`, so we return `-1`.

- **Main Loop**:
  - We read the number of test cases `t`.
  - For each test case, we read `n` and call `min_moves_to_one(n)`, then print the result.

This approach ensures that we efficiently reduce `n` to `1` using the minimum number of moves or determine if it's impossible.

['Base Case: If  is already , no moves are needed, so the answer is .',
 'Operations:',
 'Strategy:',
 'Termination Condition:',
 'Implementation:']

In [ ]:
sorted_freqs_table = sorted(heading_freqs.items(), key=lambda x: x[1], reverse=True)
sorted_freqs_table